### Запуск QtLab

Для начала надо добавить пути со скриптами и драйверами устройств, подключить библиотеки numpy и matplotlib.

In [1]:
%matplotlib qt
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)

## Параметры кубитов

In [510]:
#Параметры кубитов.all keys should be str
Qubits = {'1':{'r':{'Fr': 7.410e9, 'Frmax': 7.422e9},
               'q':{'F':
                    {'F01_max':6.102e9,
                    'F01_mid':5.368e9}, #2.306
                    'P':
                    {'nndac_coil_id':0,#0,
                     'awg_tek_coil_channel':1,
                     'coil_device_channel':1,
                    'P':None}
                    }},
        '2':{'r':{'Fr': 7.625e9, 'P':-50}, 
               'q':{'00-1-01': 4.250412e9,#4.24947e9,
                    }},
         '3':{'r':{'Fr': 8.35e9, 'P':-50}, 
               'q':{'00-1-01': 4.250412e9,#4.24947e9,
                    }}}
qubit_params=Qubits
qubits=Qubits
qubit_id = '1'
#it should be dumped too
#qubits_dump = {"qubits":qubit_params,'switch':switch_to_sample}
#qjson.dump('qubits','qubit_params',qubits_dump)


## измерительные устройства

In [ ]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
# lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')

#rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')
#awg = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
#adc = Spectrum_M3i2132('adc')
#sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
lo_ex = lo1
lo_ro = pna
#awg_tek=awg

In [3]:
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')

In [4]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')
#current_src1 = Yokogawa_GS210(address = 'GPIB1::1::INSTR')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [7]:
pna.set_sweep_mode("LIN")

True

In [8]:
CHASSIS = 0
SLOT_AWG1 = 3
SLOT_AWG2 = 5
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
awg1 = Keysight_M3202A_S('awg1', CHASSIS, SLOT_AWG1)
awg2 = Keysight_M3202A_S('awg2', CHASSIS, SLOT_AWG2)

adc = instruments.Spectrum_M4i22xx('adc')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [429]:
#Stuff for IQ-mixers opening
awg1.trigger_source_types = [6,6,6,6]
awg1.trigger_source_channels = [4000, 4000,4000,4000]
awg1.trigger_delays = [7+10,7+10,7+10,7+10] # wait 60 ns for sync with awg2
awg1.trigger_behaviours = [4,4,4,4]
awg1.stop()
for channel in range(0,3):
    awg1.set_amplitude(0.03, channel=channel)
    awg1.set_offset(0.1 ,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*50000, channel=channel)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg1.run()

#Stuff for IQ-mixers opening
awg2.trigger_source_types = [6,6,6,6]
awg2.trigger_source_channels = [4000, 4000,4000,4000]
awg2.trigger_delays = [7+10,7+10,7+10,7+10] # wait 60 ns for sync with awg2
awg2.trigger_behaviours = [4,4,4,4]
awg2.stop()
for channel in range(0,3):
    awg2.set_amplitude(0.03, channel=channel)
    awg2.set_offset(0.1 ,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform =[0]*50000, channel=channel)
awg2.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg2.run()

# Однотоновая спектроскопия резонатора в зависимости от мощности


In [421]:
lo1.set_status(0)
lo1.set_power(-5)
pna.set_status(1)
#pna.set_power(-35) #
pna.set_power(-40)
pna.set_nop(401)
pna.set_xlim(Qubits[_qubit_id]['r']['Fr']-100e6, Qubits[_qubit_id]['r']['Fr']+100e6)
pna.set_bandwidth(20)
#freqs = pna.get_freqpoints()
#S21 = pna.get_tracedata()
meas = sweep.sweep(pna, ([0], lambda x: 0, ''), filename='Однотоновая спектроскопия, образец #{}'.format(qubit_id))

HTML(value='')

Started at:  Dec 18 2018 17:27:38
Exiting sweep main thread #{} via return 10992

Elapsed time: 0 h 0 m 20.08 s


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 23069: character maps to <undefined>

In [420]:
lo1.set_status(0)
pna.set_status(1)
#pna.set_power(-35) #
powers = np.linspace(0, -60, 61)
pna.set_nop(401)
pna.set_xlim(Qubits[qubit_id]['r']['Fr']-100e6, Qubits[qubit_id]['r']['Fr']+100e6)
pna.set_bandwidth(50)
#freqs = pna.get_freqpoints()
#S21 = pna.get_tracedata()
meas = sweep.sweep(pna, (powers, pna.set_power, 'Мощность, дБ'), filename='Однотоновая спектроскопия, образец #{}'.format(qubit_id))

HTML(value='')

Started at:  Dec 18 2018 17:14:35
Exiting sweep main thread #{} via return 924

Elapsed time: 0 h 7 m 20.25 s


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 5655: character maps to <undefined>

In [8]:
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
coil_device = awg_tek

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [25]:
awg_tek.stop()
for channel in range(1,5):
    awg_tek.set_output(0, channel=channel)
    awg_tek.set_amplitude(1.0, channel=channel)
    awg_tek.set_offset(0.0,channel=channel)
    awg_tek.set_output(1, channel=channel)
    awg_tek.set_waveform([0]*50000, channel=channel)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [11]:
pna.set_status(1)

True

# Однотоновая спектроскопия резонатора в зависимости от тока


In [19]:
tone1power = -55


In [295]:
dac.set_voltage(0.04, channel=nndac_channel_id)

In [100]:
# lo1.set_status(0)
nndac_channel_id = 0
pna.set_power(-40)
pna.set_nop(101)
pna.set_bandwidth(20)
voltage_max=2
voltage_nop = 51
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(2.0,3.0,voltage_nop)
pna.set_xlim(8.1e9,8.4e9) #0-7.423e9 pi-7.415
try:
    result = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)), 
                        filename=' New anticrossings sample 1 qubit 3{} Icoil Power {}'.format(qubit_id, pna.get_power()))
finally:
    dac.set_voltage(0, channel=nndac_channel_id)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

Started at:  Dec 17 2018 08:37:16


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


KeyboardInterrupt: 

In [101]:
# lo1.set_status(0)
nndac_channel_id = 0
pna.set_power(-40)
pna.set_nop(151)
pna.set_bandwidth(30)
voltage_max=2
voltage_nop = 51
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.6,1.8,voltage_nop)
pna.set_xlim(7.325e9,7.475e9)
try:
    result = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)), 
                        filename=' New anticrossings sample 2 qubit 1{} Icoil power {}'.format(qubit_id, pna.get_power()))
finally:
    dac.set_voltage(0, channel=nndac_channel_id)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 17 2018 08:38:08


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 11260

Elapsed time: 0 h 3 m 53.34 s


In [81]:
plt.figure()
plt.pcolormesh(result['S-parameter'][1][0], result['S-parameter'][1][1], np.abs(result['S-parameter'][2]).T)
plt.colorbar()

In [79]:
plt.figure()
diff = np.gradient(result['S-parameter'][2])
plt.pcolormesh(result['S-parameter'][1][0], result['S-parameter'][1][1], np.real(diff[0]).T,vmin=-0.01, vmax=0.01)
plt.colorbar()

# Однотоновая спектроскопия резонатора от тектроникса



In [57]:
coil_device.get_status(channel=1)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


True

In [62]:
coil_device.stop

<bound method Tektronix_AWG5014.stop of <Tektronix_AWG5014.Tektronix_AWG5014 object at 0x0000024613C4BF60>>

In [65]:
coil_device.run()

In [67]:
if coil_device != awg_tek:
    raise ValueError('awg_tek is not the active coil control device. This cell is for awg_tek.')
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 1.9
current_nop = 101
#nndac_channels = [6,8,10]#[18,20]#[18, 20, 16]#[21,1,7]
awg_tek_channels = [2]
coil_device_channels = [2]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(101)
pna.set_bandwidth(50)
pna.set_power(-15) 
lo1.set_status(0)
for coil_device_channel in coil_device_channels:
    for qubit_id in ['1']:
        coil = 'coil_device_channel-'+str(coil_device_channel)
        coil_device.set_output(1, channel=coil_device_channel)
        #pna.set_nop((7.5e9-7.2e9)/301)
        pna.set_xlim(7.3e9, 7.46e9)
        result = sweep.sweep(pna, 
                        (voltages, lambda x: coil_device.set_offset(x, channel=coil_device_channel), 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename=' Tek anticrossings sample 1 qubit 1{} Icoil power {}'.format(qubit_id, pna.get_power()))
                
        #dac.set_voltage(0, channel=6)#0)
        #dac.set_voltage(0, channel=8)#2)
        #dac.set_voltage(0, channel=10)#4)
        coil_device.set_output(0, channel=coil_device_channel)
        for coil_device_channel2 in coil_device_channels:
            coil_device.set_offset(0, channel=coil_device_channel2)

coil_device.stop

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

Started at:  Dec 14 2018 08:07:17


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 12736

Elapsed time: 0 h 3 m 8.01 s


<bound method Tektronix_AWG5014.stop of <Tektronix_AWG5014.Tektronix_AWG5014 object at 0x0000024613C4BF60>>

In [ ]:
coil_device.run()

In [36]:
if coil_device != awg_tek:
    raise ValueError('awg_tek is not the active coil control device. This cell is for awg_tek.')

anticrossing_matrix = []
files_matrix_location = []
voltage_max = 2
current_nop = 401
#coil_device_channels = [1,2,3]
coil_device_channels = [1]
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
pna.set_nop(401)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']+40) 
lo1.set_status(0)
for coil_device_channel in coil_device_channels:
    anticrossing_row = []
    files_location_row = []
    for qubit_id in ['1']:
        # if we are in the main coil for the current qubit, sweep only the main coil.
        if qubits[qubit_id]['q']['P']['coil_device_channel'] == coil_device_channel:
            coil = 'coil_device_channel-('+str(coil_device_channel)+')'
            def setter(x):
                coil_device.set_offset(x, channel=coil_device_channel)
        # if we are probing a non-diagonal matrix element (non-main coil), sweep the main simultaneously.
        # this helps to identify the sign of the mutual matrix element.
        else:
            coil = 'coil_device_channel-('+str(coil_device_channel)+')+('+\
                                           str(qubits[qubit_id]['q']['P']['coil_device_channel'])+')'
            coil_device.set_output(1, channel=qubits[qubit_id]['q']['P']['coil_device_channel'])
            def setter(x):
                coil_device.set_offset(x, channel=coil_device_channel)
                coil_device.set_offset(x, channel=qubits[qubit_id]['q']['P']['coil_device_channel'])
        
        coil_device.set_output(1, channel=coil_device_channel)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        result,loc = sweep.sweep(pna, 
                        (voltages, setter, '#{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename=sample_name+' #{} anticrossings qubit {}'.format(coil, qubit_id),loc=True)
#         loc+=sample_name+' #{} anticrossings qubit {}'.format(coil, qubit_id)+'.pkl'
        files_location_row.append(loc)
        anticrossing_row.append(result['S-parameter'])
        
        
        for coil_device_channel2 in coil_device_channels:
            coil_device.set_offset(0, channel=coil_device_channel2)
    anticrossing_matrix.append(anticrossing_row)
    files_matrix_location.append(files_location_row)
lo1.set_status(1)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


KeyError: 'Fr_max'


## построение графиков задержки

In [69]:
delay_median = np.median(np.gradient(np.angle(meas['S-parameter'][2]), axis=1))
phase_compensator = np.exp(-1j*delay_median*np.arange(meas['S-parameter'][2].shape[1]))
meas_delay_compensated = meas['S-parameter'][2]*phase_compensator
meas_phase_median = np.median(np.angle(meas['S-parameter'][2]), axis=1)
#plt.figure('Однотоновая спектроскопия резонатора в зависимости от мощности')
meas_compensated = {'S-parameter':[i for i in meas['S-parameter']]}
meas_compensated['S-parameter'][2] = meas['S-parameter'][2]*phase_compensator

frequencies = meas_compensated['S-parameter'][1][1]/1e9
ang_freq_step = 2*np.pi*(frequencies[1]-frequencies[0])

#plotting.plot_measurement(meas_compensated)
plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора в зависимости от мощности, образец #{}'.format(qubit_id))
plt.pcolormesh(meas_compensated['S-parameter'][1][0], 
               meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0]/ang_freq_step, cmap='RdBu')
plt.colorbar()
#plt.close()
plt.xlabel('Мощность, дБ')
plt.ylabel('Частота, ГГц')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy.pdf')


plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора (-10 дБм), образец #{}'.format(qubit_id))
plt.plot(meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0][:,21]/ang_freq_step)
#plt.close()
plt.xlabel('Частота, ГГц')
plt.ylabel('Задержка, нс')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy-cs.pdf')

In [38]:
plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора (0 дБм), образец #{}'.format(qubit_id))
plt.plot(meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0][:,17]/ang_freq_step)
#plt.close()
plt.xlabel('Частота, ГГц')
plt.ylabel('Задержка, нс')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy-cs.pdf')

# Двухтоновая спектроскопия (определение области частот, отвечающих кубитам)

In [435]:
dac.set_voltage(0.3, channel=0)

In [178]:
pna.set_nop(100)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


True

In [436]:
#dac.set_voltage(0, channel=0)
#двухтонка в пи
ex_freqs = np.arange(3e9, 10.0e9, 4.00e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(0) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
nndac_channel_id = 0
pna.set_centerfreq(7.410e9)
pna.set_frequency(7.410e9)
#pna.set_centerfreq(7.423e9)
#pna.set_frequency(7.423e9) 7.271
#pna.set_centerfreq(7.373e9)
#pna.set_frequency(7.373e9)
pna.set_bandwidth(4)
pna.set_power(-10) #я тут меняю подожди вниуз я
#dac.set_voltage(1.695, channel=nndac_channel_id)
# dac.set_voltage(1.55, channel=nndac_channel_id)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq,drive', 'Hz'), 
                       filename = "Two-tone2 #{}".format(qubit_id))
except KeyboardInterrupt: pass 
lo1.set_status(0)

HTML(value='')

Started at:  Dec 19 2018 15:43:57
Exiting sweep main thread #{} via return 9876

Elapsed time: 0 h 6 m 6.09 s


True

In [398]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')

In [437]:
dac.set_voltage(1.775, channel=0)

In [109]:
dac.set_voltage(1.75, channel=0)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [106]:
dac.set_voltage(0, channel=0)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [444]:
#двухтонка в пи
#dac.set_voltage(1.735, channel=0)
ex_freqs = np.arange(2.27e9, 2.34e9, 0.10e6)
#ex_freqs = np.arange(0.35e9, 0.50e9, 0.02e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(5)#w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
nndac_channel_id = 0
pna.set_centerfreq(7.422e9)
pna.set_frequency(7.422e9)
#pna.set_centerfreq(7.42e9)
#pna.set_frequency(7.42e9) 
#pna.set_centerfreq(7.372e9)
#pna.set_frequency(7.372e9)
pna.set_bandwidth(2)
pna.set_power(-10) #я тут меняю подожди вниуз я
#dac.set_voltage(1.695, channel=nndac_channel_id)
# dac.set_voltage(1.55, channel=nndac_channel_id)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq,drive', 'Hz'), 
                       filename = "FineTwo-tone #{}".format(qubit_id))
except KeyboardInterrupt: pass 
lo1.set_status(0)

HTML(value='')

Started at:  Dec 19 2018 16:13:04


Exception in thread Thread-132:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 282, in main_sweep_loop
    set_single_measurement_result(mpoint, indeces)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 226, in set_single_measurement_result
    ascii_file.write(data_to_str(fmt(mpoint_m_flat))+'\n')
ValueError: I/O operation on closed file.



True

In [122]:
#Power scan
ex_freqs = np.arange(0.1e9, 0.5e9, 0.5e6)
ex_powers = np.linspace(-20,-4,33)
#lo1.set_power(-10) # no mixer
#lo1.set_power(5) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
nndac_channel_id = 0
#pna.set_centerfreq(7.3525)
#pna.set_frequency(7.3525)
pna.set_centerfreq(7.448e9)
pna.set_frequency(7.448e9)
pna.set_bandwidth(2)
pna.set_power(-30) #я тут меняю подожди вниуз я
#dac.set_voltage(1.695, channel=nndac_channel_id)
# dac.set_voltage(1.55, channel=nndac_channel_id)
header = '''#Parameter = Ex frequency'''
try:
    ex1D = sweep.sweep(pna, 
                       (ex_freqs, lo1.set_frequency, 'Second tone freq,drive', 'Hz'), 
                       (ex_powers, lo1.set_power, 'Second tone power', 'dBm'),
                       filename = "Scane Lungo Two-tone power-dependent #{}".format(qubit_id))
except KeyboardInterrupt: pass 
lo1.set_status(0)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 11 2018 07:47:29


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 6320

Elapsed time: 0 h 25 m 21.08 s


True

In [26]:
# startV = range(1.6,1.8,0.2)
# for startvoltage in startV:
# Двухтоновка в 0
ex_freqs = np.arange(0.100e9, 10.0e9, 1.0e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(0) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
#pna.set_centerfreq(7.425e9)
#pna.set_frequency(7.425e9)7.3745
pna.set_centerfreq(7.374e9)
pna.set_frequency(7.374e9)
pna.set_bandwidth(20)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-15)
# voltage_max=1.8
voltage_nop = 26
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(-0.3,-0.1,voltage_nop)
try:
    ex1D = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                        (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                        filename = "Second tone_10new  #{}, power lo {},power pna {}".format(qubit_id,\
                                                                                                          lo1.get_power(),
                                                                                                         pna.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 16 2018 05:00:30


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m
Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 282, in main_sweep_loop
    set_single_measurement_result(mpoint, indeces)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 226, in set_single_measurement_result
    ascii_file.write(data_to_str(fmt(mpoint_m_flat))+'\n')
ValueError: I/O operation on closed file.



True

In [48]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [446]:
# Духтоновка в pi
ex_freqs = np.arange(2.0e9, 10.00e9, 2.0e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(10) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
#pna.set_centerfreq(7.40e9)
#pna.set_frequency(7.40e9)
pna.set_centerfreq(7.422e9)
pna.set_frequency(7.422e9)
pna.set_bandwidth(4)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-10)
# voltage_max=1.8
voltage_nop = 11
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.72,1.82,voltage_nop)
#try:
ex1D = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                        (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                        filename = "Second tone_ #{}, power lo {},power pna {}, start voltage {}".format(qubit_id,\
                                                                                                            lo1.get_power(),
                                                                                                            pna.get_power(),
                                                                                                            voltages[1]))
#except KeyboardInterrupt: pass 
lo1.set_status(0)

HTML(value='')

Started at:  Dec 19 2018 16:20:03
Exiting sweep main thread #{} via return 3188

Elapsed time: 2 h 32 m 42.5 s


True

In [447]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')

In [448]:
# Духтоновка в 0
ex_freqs = np.arange(3.0e9, 10.0e9, 4.000e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(0) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
#pna.set_centerfreq(7.40e9)
#pna.set_frequency(7.40e9)
pna.set_centerfreq(7.410e9)
pna.set_frequency(7.410e9)
pna.set_bandwidth(4)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-10)
# voltage_max=1.8
voltage_nop = 51
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
#voltages=np.linspace(-0.2,0.3,voltage nop)
voltages=np.linspace(-0.2,0.3,voltage_nop)
#try:
ex1D = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                        (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                        filename = "Second tone_ #{}, power lo {},power pna {}, start voltage {}".format(qubit_id,\
                                                                                                            lo1.get_power(),
                                                                                                            pna.get_power(),
                                                                                                            voltages[1]))
#except KeyboardInterrupt: pass 
lo1.set_status(0)

HTML(value='')

Started at:  Dec 19 2018 18:52:50
Exiting sweep main thread #{} via return 1820

Elapsed time: 5 h 9 m 17.45 s


True

In [46]:
startV = np.arange(0.3,1.70,0.2)
for startvoltage in startV:
    #current_src1 = Yokogawa_GS210(address = 'GPIB1::1::INSTR')
    #time.sleep(10)
    
    # Двухтоновка в 0
    ex_freqs = np.arange(0.100e9, 10.0e9, 2.0e6)
    #lo1.set_power(-10) # no mixer
    lo1.set_power(5) # w/mixer
    #Измерение
    lo1.set_status(1)
    pna.set_nop(1)
    #pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
    #pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
    #pna.set_centerfreq(7.425e9)
    #pna.set_frequency(7.425e9)7.3745
    pna.set_centerfreq(7.350e9)
    pna.set_frequency(7.350e9)
    pna.set_bandwidth(20)
    #pna.set_power(Qubits[qubit_id]['r']['P'])
    pna.set_power(-15)
    # voltage_max=1.8
    voltage_nop = 26
    # voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
    voltages=np.linspace(startvoltage-0.01,startvoltage+0.2,voltage_nop)
    #try:
    ex1D = sweep.sweep(pna, 
                           (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                           (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                           filename = "Second tone_10new  #{}, power lo {},power pna {}, start voltage {}".format(qubit_id,\
                                                                                                              lo1.get_power(),
                                                                                                             pna.get_power(),
                                                                                                             voltages[1]))
    #except KeyboardInterrupt: pass 
    lo1.set_status(0)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 16 2018 14:52:16


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 3184

Elapsed time: 2 h 43 m 33.84 s


HTML(value='')

Started at:  Dec 16 2018 17:35:54


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m
Exception in thread Thread-31:
Traceback (most recent call last):
  File "C:\qtlab_replacement\qsweepy\instrument_drivers\nndac.py", line 22, in set_voltage
    self._visainstrument.ask('VOLT {:d},{:f}'.format(channel,value))
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py", line 574, in ask
    return self.query(message, delay)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyvis

KeyboardInterrupt: 

In [241]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [228]:
voltages[1]


-0.292

In [119]:
# Двухтоновка в пи
ex_freqs = np.arange(0.010e9, 1.2e9, 0.05e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(0) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.415e9)
pna.set_frequency(7.415e9)
pna.set_bandwidth(5)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-45)
voltage_max=2
voltage_nop = 61
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.65,1.80,voltage_nop)
try:
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Second tone_2  #{}, power lo {},power pna {}".format(qubit_id,\
                                                                                                          lo1.get_power(),
                                                                                                         pna.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 10 2018 09:30:46


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 11972

Elapsed time: 1 h 45 m 38.42 s


True

In [77]:
lo1.set_status(0)

True

In [47]:
#двухтонка в 0
ex_freqs = np.arange(5.0e9, 7.0e9, 5e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-5) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
pna.set_centerfreq(7.3226)
pna.set_frequency(7.3226)
pna.set_bandwidth(40)
pna.set_power(-30)
dac.set_voltage(-0.040,channel=nndac_channel_id)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq,drive', 'Hz'), 
                       filename = "Двухтоновая спектроскопия образец #{}".format(qubit_id))
except KeyboardInterrupt: pass 
lo1.set_status(0)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 07 2018 12:15:56


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1402: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1403: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1405: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 11212

Elapsed time: 0 h 2 m 12.42 s


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 277494: character maps to <undefined>

In [28]:
def find_resonator(resonator_id, criterion='max_dev_complex', delay=0, check=1, raise_on_error=True): 
#    print ('resonator_id: ', resonator_id)
    # turn of all second tones
    lo1_status = lo1.get_status()
    lo1.set_status(0)
    # prior bandwidth
    bandwidth = pna.get_bandwidth()
    pna.set_bandwidth(500)
    average = pna.get_average()
    pna.set_average(0)
    #pna.set_power(-60)
    #pna.set_nop(501)
    nop = pna.get_nop()
    xlim = pna.get_xlim()
    pna.set_nop((7.4e9-
                 7.32e9)/0.02e6+1)
    pna.set_xlim(7.32e9, 7.4e9)
    #current_src1.set_current(current)
    
    # measure S21
    results = []
    for iteration_id in range(check+1):
        S21 = pna.measure()['S-parameter']
        freqs = pna.get_points()['S-parameter'][0][1]
    
        if (criterion == 'max_dev_complex'):
            measurement = S21*np.exp(2*np.pi*1j*freqs*delay)
            #plt.figure()
            #plt.plot(freqs, np.real(measurement-np.mean(measurement)))
            #plt.plot(freqs, np.imag(measurement-np.mean(measurement)))
            #plt.plot(freqs, np.abs(measurement-np.mean(measurement)))
            results.append(freqs[np.argmax(np.abs(measurement-np.mean(measurement)))])
        if (criterion == 'min_abs'):
            measurement = np.abs(S21)
            results.append(freqs[np.argmin(measurement)])
    # if resonator is off by more 3 times tolerance, raise if raise_on_error, otherwise warn
    if (np.std(results)>qubits[resonator_id]['r']['dFr']*10):
        if raise_on_error:
            raise Exception('Could not find resonator')
        else:
            print ('Could not find resonator')
    else: 
        print('resonator found at '+str(np.mean(results)))
        
    lo1.set_status(lo1_status)
    pna.set_bandwidth(bandwidth)
    pna.set_average(average)
    pna.set_nop(nop)
    pna.set_xlim(*xlim)
    
    return np.mean(results)

In [ ]:
# Двухтоновка в пи
ex_freqs = np.arange(0.01e9, 0.8e9, 5e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-10) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.3525e9)
pna.set_frequency(7.3525e9)
pna.set_bandwidth(10)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-30)
voltage_max=2
voltage_nop = 101
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.55,1.85,voltage_nop)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Двухтоновая спектроскопия образец #{}, power {}".format(qubit_id,lo1.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)



powers = np.arange(-20, 0, 5)

delay = 70e-9
def set_volt_res():
    for _qubit_id in ['1']:
        for coil_channel_id, channel_voltage in qubits[_qubit_id]['q']['P']['Fmin_coil_currents'].items():
            #dac.set_voltage(channel_voltage, channel=nndac_channel_id)
            coil_device.set_offset(channel_voltage, channel=coil_channel_id)
        res_freq = find_resonator(qubit_id, criterion='max_dev_complex', delay=delay,
                                  check=3, raise_on_error=False)
        print(res_freq)
        pna.set_frequency(res_freq)
        pna.set_centerfreq(res_freq)
set_volt_res()
result = sweep.sweep(pna,
                     (powers,lo1.set_power,'two-tone power','dB'),
                     (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                     filename='Sample 1808_072-2 Coil #{} two-tone qubit{}'.format(1,qubit_id))

In [26]:
# Двухтоновка в пи
ex_freqs = np.arange(0.01e9, 0.8e9, 5e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-10) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.3525e9)
pna.set_frequency(7.3525e9)
pna.set_bandwidth(10)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-30)
voltage_max=2
voltage_nop = 101
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.55,1.85,voltage_nop)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Двухтоновая спектроскопия образец #{}, power {}".format(qubit_id,lo1.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 08 2018 08:22:15


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 273, in main_sweep_loop
    setter(val)
  File "<ipython-input-26-6f59aad64e09>", line 23, in <lambda>
    (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
NameError: name 'nndac_channel_id' is not defined

C:\qtlab_replacement\qsweepy\plotting.py:369: RuntimeWarning: All-NaN slice encountered
  plot.set_clim(np.nanmin(data), np.nanmax(data))


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 42604: character maps to <undefined>

In [ ]:
# Двухтоновка в нуле
ex_freqs = np.arange(0.1e9, 9e9, 5e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(0) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.3226e9)
pna.set_frequency(7.3226e9)
pna.set_bandwidth(30)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-30)
voltage_max=0.3
voltage_nop = 101
voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
#voltages=np.linspace(1.5,2.5,voltage_nop)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Двухтоновая спектроскопия образец #{}, power {}".format(qubit_id,lo1.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)

In [6]:
CHASSIS = 0
SLOT_AWG1 = 3
SLOT_AWG2 = 5

awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [7]:
awg1 = Keysight_M3202A_S('awg1', CHASSIS, SLOT_AWG1)
awg2 = Keysight_M3202A_S('awg2', CHASSIS, SLOT_AWG2)
awg1.get_clock = lambda: 1e9
awg2.get_clock = lambda: 1e9
adc = instruments.Spectrum_M4i22xx('adc')

In [23]:
# когда спектроскопия не рабтает и всё плохо, тыкай сюды!
# вектнй анлизатр цепей - в режим свипа по частоте
pna.set_sweep_mode("LIN")
# смесители - открыть
#awg_tek = awg
#for channel in range(0,4):
#awg._visainstrument.write('AWGC:DOUT1 OFF')
#awg._visainstrument.write('AWGC:DOUT2 OFF')
#awg._visainstrument.write('AWGC:DOUT3 OFF')
#awg._visainstrument.write('AWGC:DOUT4 OFF')
awg2.trigger_source_types = [0,6,6,6]
awg2.trigger_source_channels = [0, 4000,4000,4000]
awg2.trigger_delays = [30+10,7+10,7+10,7+10]
awg2.trigger_behaviours = [0,4,4,4]
awg2.set_trigger_mode(2)
awg1.set_trigger_mode(2)
awg1.trigger_source_types = [6,6,6,6]
awg1.trigger_source_channels = [4000, 4000,4000,4000]
awg1.trigger_delays = [7+10,7+10,7+10,7+10] # wait 60 ns for sync with awg2
awg1.trigger_behaviours = [4,4,4,4]
awg1.stop()
awg2.stop()
for channel in range(0,4):
    awg2.set_amplitude(0.05, channel=channel)
    awg2.set_offset(0.03,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = [0]*50000, channel=channel)
for channel in range(0,1):
    awg1.set_amplitude(0.03, channel=channel)
    awg1.set_offset(0.6,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*50000, channel=channel)
awg_tek.stop()
for channel in range(1,5):
    awg_tek.set_output(0, channel=channel)
    awg_tek.set_amplitude(0*1.0, channel=channel)
    awg_tek.set_offset(0.0,channel=channel)
    awg_tek.set_output(1, channel=channel)
    awg_tek.set_waveform([0]*50000, channel=channel)
awg2.set_marker(length=200, delay=0, channel=0, pxi_channels=1)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg2.run()
awg1.run()
#     


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [64]:
awg2.stop()
awg1.stop()

In [65]:
awg2.run()
awg1.run()

In [24]:
ex_freqs = np.arange(0.1e9, 9e9, 5e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(5) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.352e9)
pna.set_frequency(7.359e9)
pna.set_bandwidth(60)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-35)
#current_max=0.95e-3
current_nop = 451
#currents=np.linspace(current_max,-current_max,current_nop)
currents=np.linspace(-200e-3,200e-3,current_nop)
current_src.set_status(1)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, 
                       (currents, current_src.set_current, 'Coil current', 'A'),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Двухтоновая спектроскопия образец #{}, power {}".format(qubit_id,lo1.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)

A Jupyter Widget

Started at:  Sep 08 2018 01:46:43
Exiting sweep main thread #{} via return 13052

Elapsed time: 8 h 59 m 8.32 s


True

In [25]:
ex_freqs = np.arange(0.8e9, 9e9, 5e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-5) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.346e9)
pna.set_frequency(7.346e9)
pna.set_bandwidth(60)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-55)
#current_max=0.95e-3
current_nop = 481
#currents=np.linspace(current_max,-current_max,current_nop)
currents=np.linspace(-65e-6,175e-6,current_nop)
current_src.set_status(1)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, 
                       (currents, current_src.set_current, 'Coil current', 'A'),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Двухтоновая спектроскопия образец #{}, power {}".format(qubit_id,lo1.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)

A Jupyter Widget

Started at:  Sep 08 2018 10:45:55
Exiting sweep main thread #{} via return 12348

Elapsed time: 9 h 34 m 27.8 s


True

In [87]:
ex_freqs = np.arange(2e9, 12e9, 10e6)
#ex_freqs = np.arange(4.08e9, 4.12e9,0.25e6)
ex_powers = np.arange(-20, 14, 1)

pna.set_xlim(Qubits[qubit_id]['r']['Fr'], Qubits[qubit_id]['r']['Fr'])
pna.set_nop(1)
#pna.set_power(Qubits[qubit_id]['r']['P']+10)
#pna.set_power(-30)
pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_bandwidth(40)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Мощность второго тона, дБ'), 
                   (ex_freqs, lo1.set_frequency, 'Частота второго тона, ГГц'), 
                   filename = "Двухтоновая спектроскопия при различных мощностях второго (возбуждающего) тона образец #{}".format(qubit_id),\
           header = header, shuffle=True)
lo1.set_status(0)

A Jupyter Widget

Started at:  Sep 06 2018 11:52:40
Exiting sweep main thread #{} via return 1712

Elapsed time: 0 h 26 m 27.05 s


True

# Импульсы 


In [569]:
import imp
from qsweepy import pulses as pulses
imp.reload(awg_digital)
imp.reload(awg_iq_multi)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(tomography)
imp.reload(sweep)
imp.reload(pulses)
imp.reload(awg_channel)

<module 'qsweepy.awg_channel' from 'C:\\qtlab_replacement\\qsweepy\\awg_channel.py'>

In [570]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм


lo_ex.set_status(1)
lo_ro_pow = 16
lo_ex_pow=18
lo_ro.set_power(lo_ro_pow)
lo_ex.set_power(lo_ex_pow)  

pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")


marker_length = 100
#readout_trigger_delay = 185
readout_trigger_delay = 0
trg_length = 10e-9

ex_clock = 1000e6
ro_clock = 1250e6

rep_rate = 20e3 # частота повторений эксперимента
global_num_points = 50000-500

awg1.stop()
awg2.stop()
awg_tek.stop()

awg1.check_cached=True
awg2.check_cached=True
#awg_tek.set_clock(ex_clock) # клок всех авгшк
awg1.set_clock(ex_clock)
awg2.set_clock(ex_clock)
awg1.set_clock_output(True)
awg2.set_clock_output(True)
awg1.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg2.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg1.repetition_period = global_num_points
awg2.repetition_period = global_num_points

awg1.module.waveforms = [None]*4
awg1.module.AWGflush(nAWG=0) #убирает waveforms, которые были на этом канале
awg1.module.AWGflush(nAWG=1)
awg1.module.AWGflush(nAWG=2)
awg1.module.AWGflush(nAWG=3)
awg2.module.waveforms = [None]*4
awg2.module.AWGflush(nAWG=0)
awg2.module.AWGflush(nAWG=1)
awg2.module.AWGflush(nAWG=2)
awg2.module.AWGflush(nAWG=3)

# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.
#this is zashkvar
awg2.set_trigger_mode(1) #external trigger mode
awg1.set_trigger_mode(1)

awg1.trigger_source_channels = [0, 0, 0, 0]
awg2.trigger_source_channels = [0, 0, 0, 0]#=0 for external trigger
#awg2.trigger_delays = [30+10,7+10,7+10,7+10]
awg1.trigger_delays = [1,1,1,1]
awg2.trigger_delays = [1,1,1,1]

awg1.trigger_behaviours = [4,4,4,4] #Trigger is active on the falling edge
awg2.trigger_behaviours = [4,4,4,4]

awg1.trigger_source_types = [6,6,6,6] # External HARDWARE trigger per cycle
awg2.trigger_source_types = [6,6,6,6]

for channel in range(0,4):
    #awg2.set_amplitude(0.4, channel=channel)
    awg2.set_offset(0.0,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = [0]*global_num_points, channel=channel)
for channel in range(0,4):
    #awg1.set_amplitude(1.0, channel=channel)
    awg1.set_offset(0.0,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*global_num_points, channel=channel)
    
awg1.set_amplitude(1.0, channel=0)
awg1.set_amplitude(1.0, channel=1)
awg1.set_amplitude(0.2, channel=2)
awg1.set_amplitude(0.2, channel=3)

awg2.set_amplitude(0.4, channel=0)
awg2.set_amplitude(0.4, channel=1)
awg2.set_amplitude(1.0, channel=2)
awg2.set_amplitude(1.0, channel=3)
    
print(awg1.module.clockResetPhase(3, 0))
print(awg2.module.clockResetPhase(3, 0))
    
awg2.set_marker(length=100, delay=0, channel=0, pxi_channels=1)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg2.run()
awg1.run()
awg_tek.run()

awg_channels =dict()
ro_trg = awg_digital.awg_digital(awg_tek, 1)
ro_trg.mode = 'set_delay' #M3202A
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay)) #M3202A
#ro_trg.mode = 'waveform' #AWG5014C
awg_channels['ro_trg'] = ro_trg

adc.reset()
#Установить sample rate
adc.set_clock(ro_clock)
adc.set_clock_mode('pxi')

#Выбрать канал (оба)
adc.select_channel01()
adc.set_channel01_amps(200,200)

adc.set_trigger_ext0_level0(500) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(600) # logical 1 value to 600 mV
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)

adc.set_trigger_ext0()
adc.set_trigger_termination(1)
#Сегментированный режим
#adc.set_multi_mode()
#Настроить входнй диапазон в мв
#adc.init_channel01_multiple_recording()
#Настроить режим измерения
adc.set_multi_record_mode()

#Выбрать канал (оба)

0
0


In [571]:
awg_tek.stop()
awg_tek._visainstrument.write('AWGC:RMOD TRIG')
awg_tek._visainstrument.write('TRIG:WVAL LAST')
awg_tek._visainstrument.write('TRIG:IMP 1kohm')
awg_tek._visainstrument.write('TRIG:LEV 0')
awg_tek._visainstrument.write('TRIG:SLOP NEG')
awg_tek._visainstrument.write('SOUR1:ROSC:FREQ 10MHz')
awg_tek._visainstrument.write('SOUR1:ROSC:SOUR EXT')

#awg_tek.set_trigger_mode('CONT')
awg_tek.set_clock(1e9)
awg_tek.set_nop(ex_clock/rep_rate-500) # репрейт нужно задавать по=хорошему только на управляющей_t
awg_tek.check_cached = True

for channel in range(1,5):
    awg_tek.set_amplitude(1.0, channel=channel)
    awg_tek.set_offset(0.0,channel=channel)
    awg_tek.set_output(1, channel=channel)
    awg_tek.set_waveform([0]*global_num_points, channel=channel)
awg_tek.run()

awg_tek.set_digital([1]*90+[0]*(50000-300), channel=1)
awg_tek.set_digital([1]*90+[0]*(50000-300), channel=2)
for other_channels in [3,4,5,6,7,8]:
    awg_tek.set_digital([1]*1000+[0]*(50000-1210), channel=other_channels)

awg_tek._visainstrument.write('SOUR1:DEL:ADJ 398 NS')
awg_tek._visainstrument.write('SOUR2:DEL:ADJ 398 NS')
awg_tek._visainstrument.write('SOUR3:DEL:ADJ 398 NS')
awg_tek._visainstrument.write('SOUR4:DEL:ADJ 398 NS')

awg_channels =dict()
ro_trg = awg_digital.awg_digital(awg_tek, 1)
ro_trg.mode = 'set_delay' #M3202A
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay)) #M3202A
#ro_trg.mode = 'waveform' #AWG5014C
awg_channels['ro_trg'] = ro_trg

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


## Импульсный режим измерений

In [572]:
### Калибровка миксеров
#ro_if = 75e6

lo_freq = 5.2e9#5.7 e9
#lo_freq = 2.65e9#5.7 e9
lo_ro_freq = 7.6e9
_qubit_id='1'
  
    
#imp.reload(awg_iq_multi)
# промежуточные частоты для гетеродинной схемы new:

lo1.set_frequency(lo_freq)
lo_ro.set_frequency(lo_ro_freq)
#iq_ex1 = awg_iq_multi.awg_iq_multi(awg2, awg2, 0, 1, lo_ex) #M3202A
#iq_ex2 = awg_iq_multi.awg_iq_multi(awg2, awg2, 2, 3, lo_ex) #M3202A
#iq_ex3 = awg_iq_multi.awg_iq_multi(awg1, awg1, 2, 3, lo_ex) #M3202A
iq_ro = awg_iq_multi.awg_iq_multi(awg1, awg1, 0, 1, lo_ro) #M3202A
#iq_ex1.name='ex1'
#iq_ex2.name='ex2'
#iq_ex3.name='ex3'

#iq_ex1.calibration_switch_setter = lambda: rf_switch.do_set_switch(1, channel=1) if not rf_switch.do_get_switch(channel=1)==1 else None
#iq_ex2.calibration_switch_setter = lambda: rf_switch.do_set_switch(2, channel=1) if not rf_switch.do_get_switch(channel=1)==2 else None
#iq_ex3.calibration_switch_setter = lambda: rf_switch.do_set_switch(3, channel=1) if not rf_switch.do_get_switch(channel=1)==3 else None
iq_ro.calibration_switch_setter = lambda: rf_switch.do_set_switch(4, channel=1) if not rf_switch.do_get_switch(channel=1)==4 else None
iq_ex = awg_iq_multi.awg_iq_multi(awg2, awg2, 2, 3, lo_ex) #M3202A
iq_ex.calibration_switch_setter = lambda: print(rf_switch.do_set_switch(2, channel=1) if not rf_switch.do_get_switch(channel=1)==2 else None)
#iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 4, 3, lo_ro) #AWG5014C
readout_channels = {}

for tr,freq in qubits[_qubit_id]['q']['F'].items():
    iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)] = awg_iq_multi.carrier(iq_ro)
    #iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].set_frequency(qubits[_qubit_id]['r']['Fr'])
    iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].set_frequency(qubits[_qubit_id]['r']['Fr'])
    iq_ex.carriers['iq_ex_q{}'.format(_qubit_id)] = awg_iq_multi.carrier(iq_ex)
    #iq_ex.carriers['iq_ex_q{}'.format(_qubit_id)].set_frequency(qubits[_qubit_id]['q']['F']['F01_max'])
    iq_ex.carriers['iq_ex_q{}'.format(_qubit_id)].set_frequency(qubits[_qubit_id]['q']['F']['F01_mid'])
    awg_channels['iq_ro_q{}'.format(_qubit_id)] = iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)]
    awg_channels['iq_ex_q{}'.format(_qubit_id)] = iq_ex.carriers['iq_ex_q{}'.format(_qubit_id)]
    readout_channels['iq_ro_q{}'.format(_qubit_id)] = awg_channels['iq_ro_q{}'.format(_qubit_id)]
 



    #    for dc_coil_id in [1,2,3]:
    #    awg_channels['dc_coil{}'.format(dc_coil_id)] = awg_channel.awg_channel(awg_tek, dc_coil_id)
            
pg = pulses.pulses(awg_channels)
#print(ex_if*1e-6 ,lo_freq)

#print("Excitation channel", end='\n')
#iq_ex1.do_calibration(sa)
#iq_ex2.do_calibration(sa)
#iq_ex3.do_calibration(sa)
print("Readout channel", end='\n')
iq_ro.do_calibration(sa)

Readout channel
c:\data/calibrations/iq-dc/ lo_freq-7.6e+09-mixer-default
c:\data/calibrations/iq-rf/ cname-default-frequency-7.41e+09-if--1.9e+08-sideband_id-     0


In [573]:
iq_ex.do_calibration(sa)

c:\data/calibrations/iq-dc/ lo_freq-5.2e+09-mixer-default
c:\data/calibrations/iq-rf/ cname-default-frequency-5.368e+09-if-1.68e+08-sideband_id-     0


## задержка считывания

In [574]:
trigger_readout_seq = [pg.p('ro_trg', 10e-9, pg.rect, 1)]
     
adc.set_nums(50)
adc.set_nop(32768)
#adc.set_posttrigger(adc.get_nop()-4096)
if hasattr(adc, 'set_posttrigger'):
    adc.set_posttrigger(adc.get_nop()-32)
modem = modem_readout.modem_readout(pg, adc, trigger_readout_seq, axis_mean=0)
modem.readout_channels = readout_channels
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-0)) # M3201A & M3202A
ro_trg.delay = 0#Spectrum+AWG5014C
delay_calibration = modem.calibrate_delay()
readout_trigger_delay = delay_calibration[awg_channels['iq_ro_q1']]
#readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock() M3201A
readout_trigger_delay_cycles = readout_trigger_delay*ro_trg.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock())-readout_trigger_delay*adc.get_clock()) #M3201A
ro_trg.delay = int(-readout_trigger_delay_cycles)#Spectrum+AWG5014C
readout_trigger_delay_test = modem.calibrate_delay()[awg_channels['iq_ro_q1']]
#readout_trigger_delay_cycles_test = readout_trigger_delay_test*adc.get_clock() M3201A
readout_trigger_delay_cycles_test = readout_trigger_delay_test*ro_trg.get_clock()
print (readout_trigger_delay_test, readout_trigger_delay_cycles_test)

modem.calibrate_dc(amplitude=0.05)

-5.184e-07 -518.4
-6.4e-09 -6.4


{'iq_ro_q1': {'iq_calibration': array([-0.04478039-0.01218431j,  0.0307569 -0.15542356j]),
  'coherent_background': array([-0.44+0.18j, -0.4 +0.12j,  0.04+0.08j, ..., -0.32+0.16j,
         -0.3 +0.08j, -0.68+0.04j]),
  'feature': array([-0.01402349-0.16760787j,  0.10883224-0.03514168j,
          0.13974055+0.12701412j, ...,  0.14332983+0.1056059j ,
          0.07211771+0.18384173j, -0.06002339+0.10675804j])}}

## усреднённое считывание

In [575]:
modem.filters = modem.calibrated_filters
adc_reducer = data_reduce.data_reduce(modem)
adc_reducer.filters = {'S21_r{}'.format(_qubit_id): 
            data_reduce.mean_reducer(modem, 'iq_ro_q{}'.format(_qubit_id), axis=0) for _qubit_id in qubits.keys()}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

In [576]:
adc_reducers = {_qubit_id:data_reduce.data_reduce(modem) for  _qubit_id in qubits.keys()}
for _qubit_id in ['1']:
    adc_reducers[_qubit_id].filters = {'S21_r{}'.format(_qubit_id):data_reduce.mean_reducer(
        modem, 'iq_ro_q{}'.format(_qubit_id), axis=0)}
    adc_reducers[_qubit_id].extra_opts['scatter'] = True
    adc_reducers[_qubit_id].extra_opts['realimag'] = True
        
adc_reducer.filters = {'S21_r{}'.format(_qubit_id): 
            data_reduce.mean_reducer(modem, 'iq_ro_q{}'.format(_qubit_id), axis=0) for _qubit_id in ['1']}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

## длительность считывания

In [605]:
def set_adc_nop(ro_adc_length):
    adc.stop() # Spectrum
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    #print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(adc.get_nop()-64)
    
# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
    
    
ro_adc_length = 0.07e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
#ro_amplitude =0.16
#ro_dac_length = 0.08e-6
#ro_adc_length = 0.07e-6
ro_dac_length = 0.05e-6
#adc.set_nums(1000)
#adc.set_software_nums_multi(1)
adc.set_nums(100)
set_adc_nop(ro_adc_length)
adc.set_nums(100000)

## измерение среднего прошедшего импульса

In [606]:
mean_sample = data_reduce.data_reduce(adc)
mean_sample.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)

#adc_reducer_old = data_reduce.data_reduce(adc)
#adc_reducer_old.filters['Std Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage std', 0)
mean_sample.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro_q1'].get_if())
mean_sample.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro_q1'].get_if())
#adc_downsampler = data_reduce.data_reduce(adc)
#adc_downsampler.filters['Voltage'] = data_reduce.downsample_reducer(adc, 'Voltage', 0, iq_ro.get_if(), 4)
# # Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# # выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
mean_sample.extra_opts['scatter'] = True

#awg.check_cached = True

## Прохождение считывающих импульсов разной амплитуды

In [593]:
#%%mprun -f sweep.sweep
imp.reload(sweep)
imp.reload(plotting)
for _qubit_id in ['1']:
    #for coil_channel_id, channel_voltage in qubits[_qubit_id]['q']['P']['Fmin_coil_currents'].items():
        #dac.set_voltage(channel_voltage, channel=nndac_channel_id)
    #    coil_device.set_offset(channel_voltage, channel=coil_channel_id)
    mean_sample = data_reduce.data_reduce(adc)
    #print (dir(mean_sample))
    #del mean_sample.measure_deferred_result
    mean_sample.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)

    mean_sample.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].get_if())
    mean_sample.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].get_if())
    # # Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
    # # выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
    mean_sample.extra_opts['scatter'] = True

    #awg.check_cached = True

    mean_sample.filters['Std Voltage (AC)'] = data_reduce.std_reducer_noavg(adc, 'Voltage', 0, 1)
    def set_pulse_amplitude(x):
        #awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
        pg.set_seq(trigger_readout_seq+[pg.p('iq_ro_q{}'.format(_qubit_id), ro_dac_length, pg.rect, x)])
        #awg_tek.run()
    measurement = sweep.sweep(mean_sample, (np.linspace(0, 0.5, 21), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough #{}'.format(_qubit_id))
    del mean_sample.filters['Std Voltage (AC)']

HTML(value='')

Started at:  Dec 20 2018 06:02:52


Exception in thread Thread-196:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 282, in main_sweep_loop
    set_single_measurement_result(mpoint, indeces)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 226, in set_single_measurement_result
    ascii_file.write(data_to_str(fmt(mpoint_m_flat))+'\n')
ValueError: I/O operation on closed file.



KeyboardInterrupt: 

In [ ]:
zero_response = measurement['Mean Voltage (AC)'][2][0,:]
drive_amplitudes = measurement['Mean Voltage (AC)'][1][0][1:]
signal = measurement['Mean Voltage (AC)'][2][1:,:]
noise = measurement['Std Voltage (AC)'][2][1:,:]
error = noise/np.sqrt(adc.get_nums())
signal_overlap = np.sum(np.conj(signal[0,:])*signal[1:,:], axis=1)/drive_amplitudes[1:]
signal_overlap_estimate = np.real(signal_overlap[0])
signal_overlap_error = 0.5*np.sqrt(np.sum((np.abs(signal[1:,:])*error[0,:])**2, axis=1)+np.sum((np.abs(signal[0,:])*error[1:,:])**2,axis=1))/drive_amplitudes[1:]
#signal_overlap_estimate = (np.sum(np.abs(signal[0,:])**2) - np.sum(error[0,:]*np.abs(signal[0,:]))-np.sum(np.abs(error[0,:])**2))/drive_amplitudes[0]
#plt.figure()
#plt.plot(np.real(signal_overlap))
#plt.plot(np.sum(noise**2, axis=1)/adc.get_nums())
#plt.plot(signal_overlap_error)
compression = 10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))
db_compression_point = np.argmax(10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate))
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate)),-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
if np.any(db_compression_point):
    ro_amplitude = drive_amplitudes[db_compression_point+1]
else:
    ro_amplitude = drive_amplitudes[-1]
print("Readout amplitude:",ro_amplitude)    

In [ ]:
plt.figure('Компрессия считывающего импульса')
plt.plot(drive_amplitudes[1:], compression)
plt.xlabel('Амплитуда считывания')
plt.ylabel('Компрессия, дБ')

In [422]:
ro_parameters = {"adc_len":ro_adc_length,'dac_len': ro_dac_length,'amp': ro_amplitude,'adc nums':adc.nums,
                'trg_length':trg_length}
qjson.dump("setups","readout",ro_parameters)

In [624]:
ro_amplitude = 0.25
ro_sequence = trigger_readout_seq+[pg.p('iq_ro_q1', ro_dac_length, pg.rect, ro_amplitude)]

## Раби+Рамзи

In [608]:
import qsweepy.qjson as pqjson
import qsweepy.quantum_two_level_dynamics as dyn

In [609]:
imp.reload(dyn)
imp.reload(qjson)
imp.reload(data_reduce)
imp.reload(plotting)
imp.reload(fitting)
imp.reload(sweep)
imp.reload(diff_readout)

<module 'qsweepy.diff_readout' from 'C:\\qtlab_replacement\\qsweepy\\diff_readout.py'>

In [630]:
#ex_channels = ['iq_ex_00-1-01', 'iq_ex_00-1-10', 'iq_ex_01-1-02']#, 'iq_ex_00-2-02']
ex_channels = ['iq_ex_q1', 
              ]

ex_amplitudes = {'iq_ex_q1':1.0, 
                }
rabi_lengths = {'iq_ex_q1':np.arange(0., 20e-9, 1.0e-9),
                }
delaysT2_coherence = {'iq_ex_q1':np.arange(0., 20e-9, 10e-9),
                     }
ro_par = {'qubit_id':qubit_id,'ex_channels':ex_channels}
fitter = lambda x,y: fitting.fit1d(x,y,'S21_r1')
ro_par['fitter']=fitter

#rabi_lengths = np.arange(0., 2000e-9, 5e-9)
delaysT2 = np.arange(0, 20e-9, 1e-9)
target_freq_offset = -100e6
#delaysT2_coherence = np.arange(0, 5000e-9, 10e-9)
target_freq_offset_coherence = -10e6
delaysT1 = np.arange(0, 5e-6,0.2e-6)

In [598]:
imp.reload(plotting)

<module 'qsweepy.plotting' from 'C:\\qtlab_replacement\\qsweepy\\plotting.py'>

In [599]:
imp.reload(diff_readout)
diff_adc_reducer = diff_readout.diff_readout(adc_reducer)

In [600]:
adc_reducer.measure()

{'S21_r1': (-0.09917180836767767+0.7470696471205948j)}

In [601]:
plt.figure()
plt.plot(np.real(np.mean(mres['Voltage'], axis=0)))
plt.plot(np.imag(np.mean(mres['Voltage'], axis=0)))

In [631]:
twos = {ex_channel:dyn.quantum_two_level_dynamics(pg,
                                           adc_reducer,
                                           ex_channel=ex_channel,
                                           ro_channel='iq_ro_q1',
                                           ro_sequence=ro_sequence,
                                           ex_amplitude=ex_amplitudes[ex_channel],
                                           fitter=fitter, 
                                           shuffle=False,
                                           qubit_id=qubit_id,
                                           readout_measurement_name='S21_r1') for ex_channel in ex_channels}

c:\data/calibrations/setups/ readout
c:\data/calibrations/two-level-rabi-rect/ c-iq_ex_q1-f-5.368e+09-amp-r1-i0


In [632]:
#dac.set_voltage(1.775,channel=0)

In [634]:
#%%mprun -f two_diffs[0].Rabi_rect -f two_diffs[0].Ramsey -f two_diffs[0].decay -f two_diffs[0].spin_echo -f sweep.sweep -f diff_adc_reducer.measure
two_diffs = twos
imp.reload(plotting)
iters = 1
coil_voltage_range = np.linspace(0.330, 0.370, 11)

for _iter in range(iters):
    for ex_channel, two_diff in two_diffs.items():
        #for coil_voltage in coil_voltage_range:
        #dac.set_voltage(coil_voltage, channel=0)
        #two_diff.Rabi_rect(rabi_lengths[ex_channel])
        two_diff.Ramsey(delaysT2, target_freq_offset)
        #two_diff.Ramsey(delaysT2_coherence[ex_channel],target_freq_offset_coherence)
        #two_diff.decay(delaysT1, (coil_voltage_range, lambda x: dac.set_voltage(x, channel=0), 'Coil Voltage', 'V'))
        #two_diff.decay(delaysT1)
        #two_diff.spin_echo(delaysT2_coherence[ex_channel],target_freq_offset_coherence, (coil_voltage_range, lambda x: dac.set_voltage(x, channel=0), 'Coil Voltage', 'V'))
        #two_diff.Ramsey(delaysT2, target_freq_offset, (coil_voltage_range, lambda x: dac.set_voltage(x, channel=0), 'Coil Voltage', 'V'))

c:\data/calibrations/two-level-rabi-rect/ c-iq_ex_q1-f-5.368e+09-amp-r1-i0
c:\data/calibrations/two-level-ramsey/ c-iq_ex_q1-f-5.368e+09-amp-r1-i0


HTML(value='')

Started at:  Dec 20 2018 06:27:08
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 5940

Elapsed time: 0 h 2 m 7.65 s


In [558]:
awg_channels

{'ro_trg': <qsweepy.awg_digital.awg_digital at 0x1a37260d2b0>,
 'iq_ro_q1': <qsweepy.awg_iq_multi.carrier at 0x1a37260d7b8>,
 'iq_ex_q1': <qsweepy.awg_iq_multi.carrier at 0x1a37260da58>}